# 와인 리뷰 데이터 인덱싱
- 벡터 DB : PINECONE

In [18]:
from dotenv import load_dotenv
import os

load_dotenv(override=True, dotenv_path="../.env")

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")

# PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_INDEX_NAME = "wine-review2"
# PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
PINECONE_NAMESPACE = "wine-review2-namespace"

# document loader

In [6]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader('./wine_reviews/winemag-data-130k-v2.csv')
docs = loader.load()
for i, d in enumerate(docs[:3]):
    print(i, d)

0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './wine_reviews/winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red 

In [7]:
len(docs)

129971

In [17]:
from langchain_pinecone import PineconeEmbeddings

embeddings = PineconeEmbeddings(
    model="llama-text-embed-v2"
)


# Pinecone객체, index객체 생성

In [10]:
from pinecone import Pinecone, ServerlessSpec


# Pinecone 클라이언트를 초기화(객체생성)
pc = Pinecone(api_key=PINECONE_API_KEY)

- index 있으면 삭제

In [11]:
# pc.list_indexes()
# PINECONE_INDEX_NAME

# for idx in pc.list_indexes():
#     print(idx.name)
#     if idx.name == PINECONE_INDEX_NAME:
#         pc.delete_index(idx.name)

- Pinecone 객체 생성, index 생성

- 업로딩에 기간이 좀 걸림, 경우에 따라 50분 정도 걸림
- pinecone 월간 쓰기 작업 제한(Write Unit Limit)을 초과하면 더이상 업로딩을 못함

# Split 객체 생성

In [13]:
# split하기
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (예: 1000자씩 분할)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=100,
    # length_function=tiktoken_len,  # 토큰 기반 길이 측정    
    length_function=len,  # 문자수   
    separators=["\n\n", "\n", " ", ""]
    )

# 문서를 분할
chunks = text_splitter.split_documents(docs)

In [14]:
len(chunks)

129982

In [15]:
print(chunks[2])

page_content=': 2
country: US
description: Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.
designation: 
points: 87
price: 14.0
province: Oregon
region_1: Willamette Valley
region_2: Willamette Valley
taster_name: Paul Gregutt
taster_twitter_handle: @paulgwine
title: Rainstorm 2013 Pinot Gris (Willamette Valley)
variety: Pinot Gris
winery: Rainstorm' metadata={'source': './wine_reviews/winemag-data-130k-v2.csv', 'row': 2}


# 배치 크기 단위로 저장하기

- langchain_pinecone의 PineconeVectorStore으로 벡터 DB에 저장

In [ ]:
from langchain_pinecone import PineconeVectorStore

BATCH_SIZE = 500

for i in range(0, len(chunks), BATCH_SIZE):
    batch_docs = chunks[i:i + BATCH_SIZE]

    if i == 0:
        vector_store = PineconeVectorStore.from_documents(
            documents=batch_docs,
            embedding=embeddings,
            index_name=PINECONE_INDEX_NAME,
            namespace=PINECONE_NAMESPACE,
        )
    else:
        vector_store.add_documents(batch_docs)

    print(f"배치 {i//BATCH_SIZE + 1} 완료: {len(batch_docs)}개 업로드")


NotFoundError: Error code: 404 - {'error': {'message': 'The model `llama-text-embed-v2` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [7]:
# =========================
# 0. 환경 변수 로드
# =========================
from dotenv import load_dotenv
import os
import time

load_dotenv(override=True, dotenv_path="../.env")

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

PINECONE_INDEX_NAME = "wine-review2"
PINECONE_NAMESPACE = "wine-review2-namespace"


# =========================
# 1. Pinecone 초기화
# =========================
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

existing_indexes = [idx["name"] for idx in pc.list_indexes().indexes]

# =========================
# 2. 기존 인덱스 삭제 (있다면)
# =========================
if PINECONE_INDEX_NAME in existing_indexes:
    print(f"기존 인덱스 삭제 중: {PINECONE_INDEX_NAME}")
    pc.delete_index(PINECONE_INDEX_NAME)

    # 삭제 반영 대기
    time.sleep(10)

# =========================
# 3. 인덱스 재생성 (⭐ dimension=1024)
# =========================
print("새 인덱스 생성 중...")

pc.create_index(
    name=PINECONE_INDEX_NAME,
    dimension=1024,            # llama-text-embed-v2와 차원 맞춰주기
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

print("인덱스 생성 완료")

# 인덱스 준비 대기
while not pc.describe_index(PINECONE_INDEX_NAME).status["ready"]:
    print("인덱스 준비 중...")
    time.sleep(5)

print("인덱스 ready 상태")


# =========================
# 4. 문서 로드 (CSV)
# =========================
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("./wine_reviews/winemag-data-130k-v2.csv")
docs = loader.load()

print(f"원본 문서 수: {len(docs)}")


# =========================
# 5. 문서 분할
# =========================
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

chunks = text_splitter.split_documents(docs)
print(f"분할된 chunk 수: {len(chunks)}")


# =========================
# 6. Pinecone Embeddings
# =========================
from langchain_pinecone import PineconeEmbeddings

embeddings = PineconeEmbeddings(
    model="llama-text-embed-v2"
)


# =========================
# 7. Pinecone Vector Store 업로드
# =========================
from langchain_pinecone import PineconeVectorStore
import time

BATCH_SIZE = 100
SLEEP_TIME = 10  

for i in range(0, len(chunks), BATCH_SIZE):
    batch_docs = chunks[i:i + BATCH_SIZE]

    if i == 0:
        vector_store = PineconeVectorStore.from_documents(
            documents=batch_docs,
            embedding=embeddings,
            index_name=PINECONE_INDEX_NAME,
            namespace=PINECONE_NAMESPACE,
        )
    else:
        vector_store.add_documents(batch_docs)

    print(f"배치 {i // BATCH_SIZE + 1} 완료: {len(batch_docs)}개 업로드")

    time.sleep(SLEEP_TIME)

print("전체 업로드 완료")


새 인덱스 생성 중...
인덱스 생성 완료
인덱스 ready 상태
원본 문서 수: 129971
분할된 chunk 수: 129982
배치 1 완료: 100개 업로드
배치 2 완료: 100개 업로드
배치 3 완료: 100개 업로드
배치 4 완료: 100개 업로드
배치 5 완료: 100개 업로드
배치 6 완료: 100개 업로드
배치 7 완료: 100개 업로드
배치 8 완료: 100개 업로드
배치 9 완료: 100개 업로드
배치 10 완료: 100개 업로드
배치 11 완료: 100개 업로드
배치 12 완료: 100개 업로드
배치 13 완료: 100개 업로드
배치 14 완료: 100개 업로드
배치 15 완료: 100개 업로드
배치 16 완료: 100개 업로드
배치 17 완료: 100개 업로드
배치 18 완료: 100개 업로드
배치 19 완료: 100개 업로드
배치 20 완료: 100개 업로드
배치 21 완료: 100개 업로드
배치 22 완료: 100개 업로드
배치 23 완료: 100개 업로드
배치 24 완료: 100개 업로드
배치 25 완료: 100개 업로드
배치 26 완료: 100개 업로드
배치 27 완료: 100개 업로드
배치 28 완료: 100개 업로드
배치 29 완료: 100개 업로드
배치 30 완료: 100개 업로드
배치 31 완료: 100개 업로드
배치 32 완료: 100개 업로드
배치 33 완료: 100개 업로드
배치 34 완료: 100개 업로드
배치 35 완료: 100개 업로드
배치 36 완료: 100개 업로드
배치 37 완료: 100개 업로드
배치 38 완료: 100개 업로드
배치 39 완료: 100개 업로드
배치 40 완료: 100개 업로드
배치 41 완료: 100개 업로드
배치 42 완료: 100개 업로드
배치 43 완료: 100개 업로드
배치 44 완료: 100개 업로드
배치 45 완료: 100개 업로드
배치 46 완료: 100개 업로드
배치 47 완료: 100개 업로드
배치 48 완료: 100개 업로드
배치 49 완료: 100개 업로드
배치 50

PineconeApiException: (429)
Reason: Too Many Requests
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2025-04', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-cloud-trace-context': 'fc4294a007e961b0a37edba5c3a0107a', 'date': 'Mon, 05 Jan 2026 06:48:49 GMT', 'server': 'Google Frontend', 'Content-Length': '233', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"RESOURCE_EXHAUSTED","message":"Request failed. You've reached the embedding token limit (5000000) for model llama-text-embed-v2 for the current month. To continue using this model, upgrade your plan."},"status":429}
